In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

In [202]:
xy = np.loadtxt('C:\\Users\\SANHA\\Desktop\\new_data.csv', delimiter=',', dtype=np.float32)
x_data=xy[:,0:-1]
x_data = normalize(x_data, axis=0, norm='max')
y_data=xy[:,[-1]]
m_data=xy[:499,:]
r_data=xy[:1000,:]
g_data=xy[:0,:]
F_data=xy[:,:]

In [163]:
# Model Inputs
def model_inputs(real_dim, noise_dim):
    inputs_real_ = tf.placeholder(tf.float32, shape=[None, real_dim], name='inputs_real')
    inputs_z_ = tf.placeholder(tf.float32, shape=[None, noise_dim], name='inputs_z')
    
    return inputs_real_, inputs_z_

#def leaky_relu(x, alpha):
 #   return tf.maximum(alpha * x, x)

In [164]:
# Generator Network# Genera 
def model_generator(z_input, out_dim, n_units=128, reuse=False, alpha=0.01):
    # used to reuse variables, name scope
    with tf.variable_scope('generator', reuse=reuse):
        hidden_layer = tf.layers.dense(z_input, n_units, activation=tf.nn.relu)
        logits = tf.layers.dense(hidden_layer, out_dim, activation=None)
        outputs = tf.nn.sigmoid(logits)
        
        return outputs, logits

In [165]:
# Discriminator Network
def model_discriminator(input, n_units=128, reuse=False, alpha=0.1):
    with tf.variable_scope('discriminator', reuse=reuse):
        hidden_layer = tf.layers.dense(input, n_units, activation=tf.nn.relu)
        #hidden_layer = leaky_relu(hidden_layer, alpha)
        
        logits = tf.layers.dense(hidden_layer, 1, activation=None
                                )
        outputs = tf.nn.sigmoid(logits)
        
        return outputs, logits

In [166]:
input_size = 42
z_dim = 21
g_hidden_size = 128
d_hidden_size = 128
alpha = 0.1
smooth = 0.1
learning_rate = 0.001

In [167]:
tf.reset_default_graph()  # If we don't have this, as we call this block over and over, the graph gets bigger and bigger

graph = tf.Graph()
with graph.as_default():
    inputs_real, inputs_z = model_inputs(input_size, z_dim)
    
    g_outputs, g_logits = model_generator(inputs_z, input_size, n_units=g_hidden_size, reuse=False, alpha=alpha)
    
    d_outputs_real, d_logits_real = model_discriminator(inputs_real, n_units=d_hidden_size, reuse=False, alpha=alpha)
    d_outputs_fake, d_logits_fake = model_discriminator(g_outputs, n_units=d_hidden_size, reuse=True, alpha=alpha)
    
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_real, labels=tf.ones_like(d_logits_real) * (1-smooth)))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.zeros_like(d_logits_fake)))
    
    d_loss = d_loss_real + d_loss_fake
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=d_logits_fake, labels=tf.ones_like(d_logits_fake)))
    
    t_vars = tf.trainable_variables()
    g_vars = [variable for variable in t_vars if 'generator' in variable.name]
    d_vars = [variable for variable in t_vars if 'discriminator' in variable.name]
    
    # Affected Variables with var_list
    d_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(d_loss, var_list=d_vars)
    g_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(g_loss, var_list=g_vars)
    
    # Saving variables with var_list
    saver = tf.train.Saver(var_list=g_vars)

In [215]:
g_data.shape

(1100, 42)

In [209]:
samples=[]
#line_sample=[1][42]
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    f = open('C:\\Users\\SANHA\\Desktop\\gen_sample.txt', 'w')
    for step in range(500):
        
        
        batch_images = F_data[step].reshape([1, 42])
        batch_z = np.random.uniform(0, 1, size=[1, z_dim])
        
        _ = sess.run(d_optimizer, feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        _ = sess.run(g_optimizer, feed_dict={inputs_z : batch_z})
        loss_d, loss_g = sess.run([d_loss, g_loss], feed_dict={inputs_real : batch_images, inputs_z : batch_z})
        if step%100==0:
            print('step {} / {} Complete. D_Loss : {:0.3f}, G_Loss : {:0.3f}'.format(step+1, 10000, loss_d, loss_g))
        
        sample_z = np.random.uniform(0, 1, size=[1, z_dim])  # 16 Samples each epoch
        gen_samples, _ = sess.run(model_generator(inputs_z, input_size, reuse=True), feed_dict={inputs_z : sample_z})
        f.writelines((str(gen_samples)))
        temp=gen_samples[0,41]
        if temp>=0.5:
            gen_samples[0,41]=1
        else :
            gen_samples[0,41]=0
        f.writelines((str(gen_samples)))
        m_data=np.append(m_data,gen_samples,axis=0)
        g_data=np.append(g_data,gen_samples,axis=0)
       
        
    
    print('Training Complete. ')
    f.close()

step 1 / 10000 Complete. D_Loss : 1.573, G_Loss : 0.874
step 101 / 10000 Complete. D_Loss : 2.088, G_Loss : 0.973
step 201 / 10000 Complete. D_Loss : 1.848, G_Loss : 1.134
step 301 / 10000 Complete. D_Loss : 5.084, G_Loss : 1.601
step 401 / 10000 Complete. D_Loss : 9.643, G_Loss : 1.571
Training Complete. 


In [187]:
m_data.shape

(1009, 42)

In [221]:
#xy = np.loadtxt('C:\\Users\\SANHA\\Desktop\\test.csv', delimiter=',', dtype=np.float32)
#x_data=xy[:,0:-1]
#y_data=xy[:,[-1]]
#F_data=xy[:,:]


x1_data=m_data[:,0:-1]
y1_data=m_data[:,[-1]]
x2_data=r_data[:,0:-1]
y2_data=r_data[:,[-1]]
x3_data=g_data[:,0:-1]
y3_data=g_data[:,[-1]]



nb_classes=2

X=tf.placeholder(tf.float32,[None,41])
Y=tf.placeholder(tf.int32,[None,1])

Y_one_hot=tf.one_hot(Y,nb_classes)
Y_one_hot=tf.reshape(Y_one_hot,[-1,nb_classes])

W1=tf.Variable(tf.random_normal([41,128]),name='weight1')
b1=tf.Variable(tf.random_normal([128]),name='bias1')
layer1=tf.sigmoid(tf.matmul(X,W1)+b1)

W2=tf.Variable(tf.random_normal([128,128]),name='weight2')
b2=tf.Variable(tf.random_normal([128]),name='bias2')
layer2=tf.sigmoid(tf.matmul(layer1,W2)+b2)

W3=tf.Variable(tf.random_normal([128,nb_classes]),name='weight3')
b3=tf.Variable(tf.random_normal([nb_classes]),name='bias3')
logits=tf.matmul(layer2,W3)+b3
hypothesis=tf.nn.softmax(logits)

cost_i=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y_one_hot)

cost=tf.reduce_mean(cost_i)
optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

prediction=tf.argmax(hypothesis,1) #가능성을 퍼센트로~~
correct_prediction=tf.equal(prediction,tf.arg_max(Y_one_hot,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(100):
        sess.run(optimizer,feed_dict={X:x_data,Y:y_data})
        #print("training by gan sample")
        if step %10==0:
            loss,acc=sess.run([cost,accuracy],feed_dict={X:x_data,Y:y_data})
            print("step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step,loss,acc))
  
    tr=0
    fa=0
    total=0


    pred = sess.run(prediction, feed_dict={X: x1_data})
    for p, y in zip(pred, y1_data.flatten()):
            if(p==int(y)):
                tr=tr+1
            else:
                fa=fa+1
            #print("[{}] Prediction: {} Real Y: {}".format(p == int(y), p, int(y)))

    print("true={} false: {} acc: {:0.2f}%".format(tr,fa,tr/(tr+fa)))

step:     0	Loss: 2.885	Acc: 25.55%
step:    10	Loss: 1.679	Acc: 32.22%
step:    20	Loss: 0.978	Acc: 59.07%
step:    30	Loss: 0.685	Acc: 76.56%
step:    40	Loss: 0.556	Acc: 82.32%
step:    50	Loss: 0.488	Acc: 85.60%
step:    60	Loss: 0.447	Acc: 87.62%


KeyboardInterrupt: 